In [1]:
import copy
import os
from dataclasses import dataclass, field
import sys
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
import torch
import torch.nn.functional as F
from diffusers import AutoencoderKL, DDPMScheduler
from diffusers.optimization import get_scheduler
from einops import rearrange
from tqdm.auto import tqdm
from transformers import AutoTokenizer, CLIPTextModel
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from accelerate import Accelerator

# 假设我们有DiT模型定义，类似于diffusers中的UNet
# from my_models import DiTModel, DiTConfig # 这是一个假设的模型定义
from diffusers import HunyuanVideoPipeline, HunyuanVideoTransformer3DModel, AutoencoderKLHunyuanVideo

/u/jcai2/miniconda3/envs/fastvideo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
/u/jcai2/miniconda3/envs/fastvideo/lib/python3.11/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.5.1+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.10 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [11]:
from diffusers import FlowMatchEulerDiscreteScheduler
import json

# 读取 scheduler_config.json
scheduler_config_path = "scheduler_config.json"
with open(scheduler_config_path, "r") as f:
    scheduler_config = json.load(f)

# 用配置初始化 scheduler
scheduler = FlowMatchEulerDiscreteScheduler.from_config(scheduler_config)

In [24]:
from diffusers import HunyuanVideoPipeline, HunyuanVideoTransformer3DModel, AutoencoderKLHunyuanVideo
vae = AutoencoderKLHunyuanVideo.from_config("HunyuanConfig/vae.json")

In [ ]:
def _fm_from_pred_velocity_to_pred_video(model_output, noisy_latents, timestep):
    """
    from velocity to video
    """
    sigmas = scheduler.sigmas
    schedule_timesteps = scheduler.timesteps
    step_indices = [scheduler.index_for_timestep(t, schedule_timesteps) for t in timestep]
    sigma = sigmas[step_indices].flatten()
    
    # 调整 sigma 的维度用于广播
    while len(sigma.shape) < len(noisy_latents.shape):
        sigma = sigma.unsqueeze(-1)
        
    # x_clean_pred = x_t + t * v_pred
    pred_video = noisy_latents + sigma * model_output
    
    return pred_video

In [22]:
print(scheduler.sigmas)

tensor([1.0000, 0.9999, 0.9997, 0.9996, 0.9994, 0.9993, 0.9991, 0.9990, 0.9988,
        0.9987, 0.9986, 0.9984, 0.9983, 0.9981, 0.9980, 0.9978, 0.9977, 0.9975,
        0.9974, 0.9972, 0.9971, 0.9969, 0.9968, 0.9966, 0.9965, 0.9964, 0.9962,
        0.9961, 0.9959, 0.9958, 0.9956, 0.9955, 0.9953, 0.9951, 0.9950, 0.9948,
        0.9947, 0.9945, 0.9944, 0.9942, 0.9941, 0.9939, 0.9938, 0.9936, 0.9935,
        0.9933, 0.9932, 0.9930, 0.9928, 0.9927, 0.9925, 0.9924, 0.9922, 0.9921,
        0.9919, 0.9918, 0.9916, 0.9914, 0.9913, 0.9911, 0.9910, 0.9908, 0.9906,
        0.9905, 0.9903, 0.9902, 0.9900, 0.9898, 0.9897, 0.9895, 0.9894, 0.9892,
        0.9890, 0.9889, 0.9887, 0.9885, 0.9884, 0.9882, 0.9881, 0.9879, 0.9877,
        0.9876, 0.9874, 0.9872, 0.9871, 0.9869, 0.9867, 0.9866, 0.9864, 0.9862,
        0.9861, 0.9859, 0.9857, 0.9856, 0.9854, 0.9852, 0.9851, 0.9849, 0.9847,
        0.9845, 0.9844, 0.9842, 0.9840, 0.9839, 0.9837, 0.9835, 0.9833, 0.9832,
        0.9830, 0.9828, 0.9826, 0.9825, 

In [18]:
model_output = torch.randn((1, 3, 16, 224, 224))
noisy_latents = torch.randn((1, 3, 16, 224, 224))

def _sample_timestep() -> torch.Tensor:
    """
    随机选择一个时间步，用于生成噪声，返回形状为 [batch_size] 的 tensor。
    """
    idx = torch.randint(0, len(scheduler.timesteps), (1,))
    sampled_timesteps = scheduler.timesteps[idx]
    return sampled_timesteps
timestep = _sample_timestep()
print(timestep)
x = _fm_from_pred_velocity_to_pred_video(model_output, noisy_latents, list(timestep))

tensor([846.6813])
